In [1]:
import numpy as np
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, ErrorCode, Variable, ParcelsRandom, DiffusionUniformKh, AdvectionDiffusionM1, AdvectionDiffusionEM
from parcels.tools.converters import GeographicPolar, Geographic
from datetime import timedelta
from glob import glob
import math
import xarray as xr

In [2]:
# Particle set
"""
This sets up the oil particle with different variables used in the kernels.

The water fraction is the amount of water in the oil droplets from emulsification, and affects the density and the viscosity.
The evaporation fraction affects the viscosity of the oil.
"""

# Norman Wells Light Crude Oil
class LowDensityOil(ScipyParticle):
    viscosity = Variable('viscosity', dtype=np.float32, initial=0.007)
    density = Variable('density', dtype=np.float32, initial=838)
    mass = Variable('mass', dtype=np.float32, initial=838)
    original_mass = Variable('original_mass', dtype=np.float32, initial=838)
    original_density = Variable('original_density', dtype=np.float32, initial=838)
    original_viscosity = Variable('original_viscosity', dtype=np.float32, initial=0.007)
    water_fraction = Variable('water_fraction', dtype=np.float32, initial=0)
    max_water_fraction = Variable('max_water_fraction', dtype=np.float32, initial=0.9)
    fraction_evaporated = Variable('fraction_evaporated', dtype=np.float32, initial=0)
    is_emulsified = Variable('is_emulsified', dtype=np.float32, initial=0)
    emulsification_time = Variable('emulsification_time', dtype=np.float32, initial=0)
    vapor_pressure = Variable('vapor_pressure', dtype=np.float32, initial=119933.00831083095)
    interfacial_tension = Variable('interfacial_tension', dtype=np.float32, initial=0.0248)
    area = Variable('area', dtype=np.float32, initial = 0.1)
    interfacial_area = Variable('interfacial_area', dtype=np.float32, initial = 0)
    d_min = Variable('d_min', dtype=np.float32, initial = 1e-6)
    d_max = Variable('d_max', dtype=np.float32, initial = 1e-5)
    max_diameter = Variable('max_diameter', dtype=np.float32, initial = 1e-4)
    min_diameter = Variable('min_diameter', dtype=np.float32, initial = 1e-4)
    rise_velocity = Variable('rise_velocity', dtype=np.float32, initial = 0)
    U10 = Variable('U10', dtype=np.float32, initial = 0)
    V10 = Variable('V10', dtype=np.float32, initial = 0)
    wind_speed = Variable('wind_speed', dtype=np.float32, initial = 0)
    entrainment_time = Variable('entrainment_time', dtype=np.float32, initial = 0)

# Dos Cuadros HE-26 [2011] Medium Crude Oil
class MediumDensityOil(ScipyParticle):
    viscosity = Variable('viscosity', dtype=np.float32, initial=0.021)
    density = Variable('density', dtype=np.float32, initial=879)
    mass = Variable('mass', dtype=np.float32, initial=879)
    original_mass = Variable('original_mass', dtype=np.float32, initial=879)
    original_density = Variable('original_density', dtype=np.float32, initial=879)
    original_viscosity = Variable('original_viscosity', dtype=np.float32, initial=0.021)
    water_fraction = Variable('water_fraction', dtype=np.float32, initial=0)
    max_water_fraction = Variable('max_water_fraction', dtype=np.float32, initial=0.9)
    fraction_evaporated = Variable('fraction_evaporated', dtype=np.float32, initial=0)
    is_emulsified = Variable('is_emulsified', dtype=np.float32, initial=0)
    emulsification_time = Variable('emulsification_time', dtype=np.float32, initial=0)
    vapor_pressure = Variable('vapor_pressure', dtype=np.float32, initial=26155.627736028568)
    interfacial_tension = Variable('interfacial_tension', dtype=np.float32, initial=0.0161)
    area = Variable('area', dtype=np.float32, initial = 0.1)
    interfacial_area = Variable('interfacial_area', dtype=np.float32, initial = 0)
    d_min = Variable('d_min', dtype=np.float32, initial = 1e-6)
    d_max = Variable('d_max', dtype=np.float32, initial = 1e-5)
    max_diameter = Variable('max_diameter', dtype=np.float32, initial = 1e-4)
    min_diameter = Variable('min_diameter', dtype=np.float32, initial = 1e-4)
    rise_velocity = Variable('rise_velocity', dtype=np.float32, initial = 0)
    U10 = Variable('U10', dtype=np.float32, initial = 0)
    V10 = Variable('V10', dtype=np.float32, initial = 0)
    wind_speed = Variable('wind_speed', dtype=np.float32, initial = 0)
    entrainment_time = Variable('entrainment_time', dtype=np.float32, initial = 0)
    decay = Variable('decay', dtype=np.float32, initial = 0)

# Platform Ellen A038 Heavy Crude Oil    
class HighDensityOil(ScipyParticle):
    viscosity = Variable('viscosity', dtype=np.float32, initial=3.1)
    density = Variable('density', dtype=np.float32, initial=959)
    mass = Variable('mass', dtype=np.float32, initial=959)
    original_mass = Variable('original_mass', dtype=np.float32, initial=959)
    original_density = Variable('original_density', dtype=np.float32, initial=959)
    original_viscosity = Variable('original_viscosity', dtype=np.float32, initial=3.1)
    water_fraction = Variable('water_fraction', dtype=np.float32, initial=0)
    max_water_fraction = Variable('max_water_fraction', dtype=np.float32, initial=0.9)
    fraction_evaporated = Variable('fraction_evaporated', dtype=np.float32, initial=0)
    is_emulsified = Variable('is_emulsified', dtype=np.float32, initial=0)
    emulsification_time = Variable('emulsification_time', dtype=np.float32, initial=0)
    vapor_pressure = Variable('vapor_pressure', dtype=np.float32, initial=48026.29949199871)
    interfacial_tension = Variable('interfacial_tension', dtype=np.float32, initial=0.0223)
    area = Variable('area', dtype=np.float32, initial = 0.1)
    interfacial_area = Variable('interfacial_area', dtype=np.float32, initial = 0)
    d_min = Variable('d_min', dtype=np.float32, initial = 1e-6)
    d_max = Variable('d_max', dtype=np.float32, initial = 1e-5)
    max_diameter = Variable('max_diameter', dtype=np.float32, initial = 1e-4)
    min_diameter = Variable('min_diameter', dtype=np.float32, initial = 1e-4)
    rise_velocity = Variable('rise_velocity', dtype=np.float32, initial = 0)
    U10 = Variable('U10', dtype=np.float32, initial = 0)
    V10 = Variable('V10', dtype=np.float32, initial = 0)
    wind_speed = Variable('wind_speed', dtype=np.float32, initial = 0)
    entrainment_time = Variable('entrainment_time', dtype=np.float32, initial = 0)

In [3]:
# Add MOI field set

data_path = '/storage/shared/oceanparcels/input_data/MOi/'
ufiles = sorted(glob(data_path+'psy4v3r1/psy4v3r1-daily_U_2019-06-12.nc'))
vfiles = [f.replace('_U_', '_V_') for f in ufiles]
wfiles = [f.replace('_U_', '_W_') for f in ufiles]
mesh_mask = data_path + 'domain_ORCA0083-N006/coordinates.nc'

filenames = {'U': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': ufiles},
             'V': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': vfiles},
             'W': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': wfiles}}
variables = {'U': 'vozocrtx', 'V': 'vomecrty', 'W': 'vovecrtz'}
dimensions = {'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}}

fieldset_moi = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation = True)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [4]:
open_fieldset_moi = xr.open_dataset(data_path+'psy4v3r1/psy4v3r1-daily_U_2019-06-12.nc')
open_fieldset_moi

<xarray.Dataset>
Dimensions:       (y: 3059, x: 4322, deptht: 50)
Coordinates:
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
  * x             (x) int32 1 2 3 4 5 6 7 ... 4316 4317 4318 4319 4320 4321 4322
  * y             (y) int32 1 2 3 4 5 6 7 ... 3053 3054 3055 3056 3057 3058 3059
    time_counter  datetime64[ns] ...
  * deptht        (deptht) float64 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
Data variables:
    sotkeavmu1    (y, x) float32 ...
    sotkeavmu15   (y, x) float32 ...
    sotkeavmu30   (y, x) float32 ...
    sotkeavmu50   (y, x) float32 ...
    vozocrtx      (deptht, y, x) float32 ...
Attributes:
    Conventions:  CF-1.0
    file_name:    ORCA12_LIM-T00_y2021m09d27_gridU.nc
    institution:  MERCATOR OCEAN
    source:       NEMO
    TimeStamp:    2021-OCT-03 18:27:01 GMT-0000
    references:   http://www.mercator-ocean.eu

In [5]:
# Add wind fieldset from netcdf file

wind_dataset = xr.open_dataset('wind_data_12_june_2016_after5.nc')

filenames_t_wind = {'U': 'wind_data_12_june_2016_after5.nc', 'V': 'wind_data_12_june_2016_after5.nc'}
variables_t_wind = {'U': 'u10', 'V': 'v10'}
dimensions_t_wind = {'U': {'lat': 'latitude', 'lon': 'longitude', 'time': 'time'},
              'V': {'lat': 'latitude', 'lon': 'longitude', 'time': 'time'}}

fieldset_wind = FieldSet.from_netcdf(filenames_t_wind, variables_t_wind, dimensions_t_wind)

In [6]:
wind_dataset

<xarray.Dataset>
Dimensions:    (longitude: 9, latitude: 17, time: 19)
Coordinates:
  * longitude  (longitude) float32 1.0 1.25 1.5 1.75 2.0 2.25 2.5 2.75 3.0
  * latitude   (latitude) float32 62.0 61.75 61.5 61.25 ... 58.5 58.25 58.0
  * time       (time) datetime64[ns] 2016-06-12T05:00:00 ... 2016-06-12T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-05-28 11:19:07 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

In [7]:
# Add wind speed fields to fieldset

u_wind_factor = Field('U10_factor', wind_dataset['u10'] * 0, grid=fieldset_wind.U.grid)
v_wind_factor = Field('V10_factor', wind_dataset['v10'] * 0, grid=fieldset_wind.V.grid)

u_wind_total = Field('U10', wind_dataset['u10'], grid=fieldset_wind.U.grid)
v_wind_total = Field('V10', wind_dataset['v10'], grid=fieldset_wind.V.grid)

fieldset_moi.add_field(u_wind_factor)
fieldset_moi.add_field(v_wind_factor)
fieldset_moi.add_field(u_wind_total)
fieldset_moi.add_field(v_wind_total)

fieldset_moi.U10_factor.units = GeographicPolar()
fieldset_moi.V10_factor.units = Geographic()

fieldset_moi.add_constant('density_water', 1028)
fieldset_moi.add_constant('viscosity_water', 0.00122/1028)
fieldset_moi.add_constant('g', 9.81)
fieldset_moi.add_constant('wind_threshold', 5)
fieldset_moi.add_constant('sea_surface_temperature', 283)
fieldset_moi.add_constant('mld', 50)
fieldset_moi.add_constant('density_air', 1.22)
fieldset_moi.add_constant('wave_age', 35)
fieldset_moi.add_constant('surface_z', 0)
fieldset_moi.add_constant('v_k', 0.4)
fieldset_moi.add_constant('theta', 1)
fieldset_moi.add_constant('phi', 0.9)

In [8]:
# Delete particles if out of bounds

def DeleteParticle(particle, fieldset, time):
    particle.delete()

In [9]:
# Oil advection kernel with wind factor added
    
def OilAdvectionRK4(particle, fieldset, time):
    
    if particle.depth < 0.001:
        (u1, v1) = fieldset.UV[particle]
        u10_1 = fieldset.U10_factor[particle]
        v10_1 = fieldset.V10_factor[particle]
        lon1, lat1 = (particle.lon + (u1 + u10_1)*.5*particle.dt, particle.lat + (v1 + v10_1)*.5*particle.dt)
        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1, particle]
        u10_2 = fieldset.U10_factor[time + .5 * particle.dt, particle.depth, lat1, lon1, particle]
        v10_2 = fieldset.V10_factor[time + .5 * particle.dt, particle.depth, lat1, lon1, particle]
        lon2, lat2 = (particle.lon + (u2 + u10_2)*.5*particle.dt, particle.lat + (v2 + v10_2)*.5*particle.dt)
        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2, particle]
        u10_3 = fieldset.U10_factor[time + .5 * particle.dt, particle.depth, lat2, lon2, particle]
        v10_3 = fieldset.V10_factor[time + .5 * particle.dt, particle.depth, lat2, lon2, particle]
        lon3, lat3 = (particle.lon + (u3 + u10_3)*particle.dt, particle.lat + (v3 + v10_3)*particle.dt)
        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3, particle]
        u10_4 = fieldset.U10_factor[time + particle.dt, particle.depth, lat3, lon3, particle]
        v10_4 = fieldset.V10_factor[time + particle.dt, particle.depth, lat3, lon3, particle]
        particle.lon += ((u1 + u10_1) + 2*(u2 + u10_2) + 2*(u3 + u10_3) + (u4 + u10_4)) / 6. * particle.dt
        particle.lat += ((v1 + v10_1) + 2*(v2 + v10_2) + 2*(v3 + v10_3) + (v4 + v10_4)) / 6. * particle.dt
        
    else:
        (u1, v1) = fieldset.UV[particle]
        lon1, lat1 = (particle.lon + u1*.5*particle.dt, particle.lat + v1*.5*particle.dt)
        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1, particle]
        lon2, lat2 = (particle.lon + u2*.5*particle.dt, particle.lat + v2*.5*particle.dt)
        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2, particle]
        lon3, lat3 = (particle.lon + u3*particle.dt, particle.lat + v3*particle.dt)
        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3, particle]
        particle.lon += (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
        particle.lat += (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt


In [10]:
# Entrainment

def Entrainment(particle, fieldset, time):
    
    particle.U10 = fieldset.U10[particle]
    particle.V10 = fieldset.V10[particle]
    particle.wind_speed = math.sqrt(particle.U10**2 + particle.V10**2)
    
    if particle.depth < 0.001:
        
        d_0 = 4 * math.sqrt(particle.interfacial_tension / (fieldset.g*(fieldset.density_water - particle.density)))
        weber = fieldset.density_water * fieldset.g * 0.0246 * particle.wind_speed**2 * d_0 / particle.interfacial_tension
        ohns = (particle.viscosity * particle.density) / (math.sqrt(particle.density * particle.interfacial_tension * d_0))
        if particle.wind_speed > 5:
            T_p = 2*math.pi/(0.877 * fieldset.g * (1/(1.17*particle.wind_speed)))
            F_bw = 0.032 * (particle.wind_speed - 5)/T_p
        else:
            F_bw = 0
            
        entrainment_rate = 4.604e-10 * weber**1.805 * ohns**(-1.023) * F_bw
        if (entrainment_rate * particle.dt) < 0.01:
            entrainment_probability = entrainment_rate * particle.dt
        else:
            entrainment_probability = 1 - math.exp(-1 * entrainment_rate * particle.dt)
        
        if ParcelsRandom.random() < entrainment_probability:
            particle.depth = 1.5 * 0.0246 * particle.wind_speed**2
    
    elif particle.depth > 0.001 and particle.entrainment_time == 0:
        particle.entrainment_time += 1

In [11]:
# Turbulent Mixing

def KPP_wind_mixing(particle, fieldset, time):
    """
    Author: Victor Onink, 25/01/22
    """
    if particle.depth > 0.001 and particle.entrainment_time != 0:
        # Loading the mixed layer depth from the fieldset
        mixed_layer_depth = fieldset.mld

        # Below the MLD there is no wind-driven turbulent diffusion according to KPP theory, so we set both Kz and dKz to zero.
        if particle.depth > mixed_layer_depth:
            Kz = 0
            dKz = 0

        # Within the MLD we compute the vertical diffusion according to Boufadel et al. (2020) https://doi.org/10.1029/2019JC015727
        else:
          # Calculate the wind speed at the ocean surface
            w_10 = particle.wind_speed

          # Drag coefficient according to Large & Pond (1981) https://doi.org/10.1175/1520-0485(1981)011%3C0324:OOMFMI%3E2.0.CO;2
            C_D = min(max(1.2E-3, 1.0E-3 * (0.49 + 0.065 * w_10)), 2.12E-3)

          # Calculate the surface wind stress based on the surface wind speed and the density of air
            tau = C_D * fieldset.density_air * w_10 ** 2

          # Calcuate the friction velocity of water at the ocean surface using the surface wind stress and the surface water density
            U_W = math.sqrt(tau / fieldset.density_water)

          # Calcuate the surface roughness z0 following Zhao & Li (2019) https://doi.org/10.1007/s10872-018-0494-9
            z0 = 3.5153e-5 * fieldset.wave_age ** (-0.42) * w_10 ** 2 / fieldset.g

          # The corrected particle depth, since the depth is not always zero for the surface circulation data
            z_correct = particle.depth - fieldset.surface_z

          # The diffusion gradient at particle.depth
            C1 = (fieldset.v_k * U_W * fieldset.theta) / (fieldset.phi * mixed_layer_depth ** 2)
            dKz = C1 * (mixed_layer_depth - z_correct) * (mixed_layer_depth - 3 * z_correct - 2 * z0)

          # The KPP profile vertical diffusion, at a depth corrected for the vertical gradient in Kz
            C2 = (fieldset.v_k * U_W * fieldset.theta) / fieldset.phi
            Kz = C2 * (z_correct + z0) * math.pow(1 - z_correct / mixed_layer_depth, 2)

        # The Markov-0 vertical transport from Grawe et al. (2012) http://dx.doi.org/10.1007/s10236-012-0523-y
        gradient = dKz * particle.dt
        R = ParcelsRandom.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3) * math.sqrt(2 * Kz)

        # Update the particle depth
        particle.depth = particle.depth + gradient + R
        if particle.depth < 0:
            particle.depth = particle.depth * -1

In [12]:
# Buyonacy velocity rise

def Buoyancy(particle, fieldset, time):
    
    if particle.depth > 0.001 and particle.entrainment_time != 0:
        
        r = particle.min_diameter
        particle.rise_velocity = 2 * 9.81 * (1 - (particle.density/fieldset.density_water)) * r**2 / (9 * fieldset.viscosity_water)

        if (particle.rise_velocity * 2 * r / fieldset.viscosity_water) > 50:
            particle.rise_velocity = math.sqrt((16/3) * 9.81 * (1 - (particle.density/fieldset.density_water)) * r)

        particle.depth = particle.depth - particle.rise_velocity * particle.dt

        if particle.depth < 0.001:
            particle.depth = 0
            particle.entrainment_time = 0

In [13]:
number_particles = 100

lonp = 2.4757442009520174 * np.ones(number_particles)
latp = 59.876814891293556 * np.ones(number_particles)
depp = np.zeros(number_particles)

In [ ]:
pset_medium = ParticleSet.from_list(fieldset=fieldset_moi, pclass=MediumDensityOil, lon=lonp, lat=latp, depth=depp)

kernels_medium = pset_medium.Kernel(OilAdvectionRK4) + pset_medium.Kernel(Entrainment) + pset_medium.Kernel(KPP_wind_mixing) + pset_medium.Kernel(Buoyancy)

output_file_medium = pset_medium.ParticleFile(name="test-experiment", outputdt=timedelta(minutes=30))

pset_medium.execute(kernels_medium, runtime=timedelta(hours=2), dt=timedelta(seconds=30), recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle}, output_file=output_file_medium)

output_file_medium.close()